# Анализ оттока клиентов фитнесс-клуба «Культурист-датасаентист» 

Описание проекта

Сеть фитнес-центров «Культурист-датасаентист» разрабатывает стратегию взаимодействия с клиентами на основе аналитических данных.
Распространённая проблема фитнес-клубов и других сервисов — отток клиентов. Как понять, что клиент больше не с вами? Можно записать в отток тех, кто попросил закрыть договор или удалил аккаунт. Однако клиенты не всегда уходят демонстративно: чаще перестают пользоваться сервисом тихо.
Индикаторы оттока зависят от специфики отрасли. Когда пользователь редко, но стабильно закупается в интернет-магазине — не похоже, что он «отвалился». А вот если две недели не заходит на канал с ежедневно обновляемым контентом, дела плохи: подписчик заскучал и, кажется, оставил вас.
Для фитнес-центра можно считать, что клиент попал в отток, если за последний месяц ни разу не посетил спортзал. Конечно, не исключено, что он уехал на Бали и по приезде обязательно продолжит ходить на фитнес. Однако чаще бывает наоборот. Если клиент начал новую жизнь с понедельника, немного походил в спортзал, а потом пропал — скорее всего, он не вернётся.
Чтобы бороться с оттоком, отдел по работе с клиентами «Культуриста-датасаентиста» перевёл в электронный вид множество клиентских анкет. Ваша задача — провести анализ и подготовить план действий по удержанию клиентов. 
    
Данные клиента за предыдущий до проверки факта оттока месяц:

'gender' — пол;

'Near_Location' — проживание или работа в районе, где находится фитнес-центр;

'Partner' — сотрудник компании-партнёра клуба (сотрудничество с компаниями, чьи сотрудники могут получать скидки на абонемент — в таком случае фитнес-центр хранит информацию о работодателе клиента);

Promo_friends — факт первоначальной записи в рамках акции «приведи друга» (использовал промо-код от знакомого при оплате первого абонемента);

'Phone' — наличие контактного телефона;

'Age' — возраст;


'Lifetime' — время с момента первого обращения в фитнес-центр (в месяцах).
    Информация на основе журнала посещений, покупок и информация о текущем статусе абонемента клиента:


'Contract_period' — длительность текущего действующего абонемента (месяц, 6 месяцев, год);

'Month_to_end_contract' — срок до окончания текущего действующего абонемента (в месяцах);

'Group_visits' — факт посещения групповых занятий;

'Avg_class_frequency_total' — средняя частота посещений в неделю за все время с начала действия абонемента;

'Avg_class_frequency_current_month' — средняя частота посещений в неделю за предыдущий месяц;

'Avg_additional_charges_total' — суммарная выручка от других услуг фитнес-центра: кафе, спорттовары, косметический и массажный салон.

'Churn' — факт оттока в текущем месяце.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sbn
from scipy import stats as st
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
import plotly.graph_objects as go
from sklearn.metrics import silhouette_score


## Загрузка данных

### Просмотр данных

In [ ]:
fitness_data=pd.read_csv('/datasets/gym_churn.csv')

In [ ]:
fitness_data.head()

In [ ]:
fitness_data.info()

Типы данных соответствуют своим значениям. Названия колонок не соответствуют принятым стандартам, переведем их в нижний регистр

In [ ]:
fitness_data.columns=fitness_data.columns.str.lower()

In [ ]:
#Проверяем  наши изменения
fitness_data.head()

Выводы:
    
Данные отброжаются нормально. Все данные представлены в числовом виде. Это очень удобно для расчетов. Типы данных соответствуют своим значениям. Переходим к исследовательскому анализу  

## Исследовательский анализ данных (EDA)

### Поиск пропущенных значений

In [ ]:
fitness_data.isna().sum()

Пропущенные значения не обнаружены

### Поиск дубликатов

In [ ]:
fitness_data.duplicated().sum()

Дубликаты не обнаружены

### Просмотр средних значений признаков и стадартного отклонения 

In [ ]:
fitness_data.describe()

Стандартное отклонение везде меньше или примерно равно среднему значению признаков. Это говорит, о том что в данных нет  аномалий или выбросов.

### Средние значения признаков в двух группах — тех, кто ушел в отток и тех, кто остался

In [ ]:
fitness_data.groupby('churn').mean()


Если средние значения равны, то признак не влияет на отток, и наоборот. В данном случае получается на отток не влияют признаки пол и телефон, остальные влияют.

### Столбчатые гистограммы и распределения признаков  по группам, кто ушёл (отток) и тех, кто остался 

In [ ]:
for index in fitness_data.columns[:-1]:
    plt.figure(figsize=(35, 15))
    plt.rcParams['font.size'] = '30'
    sbn.histplot(x=index, hue='churn', data=fitness_data)
    plt.title(f'Соотношение значений признака {index} по группам')
    plt.ylabel(f'{index}')
    plt.show()

In [ ]:
fitness_data.head()

Колонка gender - мужчин оставшихся больше в 2,5 раза чем женщин, в оттоке примерно такое же соотношение .

Колонка near_location - клиентов значительно больше, которые живут  или работают рядом с клубом. Постоянных клиентов в несколько раз больше, чем в оттоке.

Колонка partner - сотрудников компании-партнёра немного больше, чем простых клиентов. Оставшихся клиентов, работающих в компании-партнере в несколько раз больше других клиентов, а в оттоке больше клиентов без партнерства, чем клиентов с партнерством.

Колонка promo_friends - клиентов без промокода больше, чем с промокодом. Но в группе с промокодом отток в несколько раз меньше, чем в группе без промокода.

Колонка phone - количество клиентов, оставивших свой телефон в записи, значительно больше, чем тех кто не оставил. Отток в несколько раз меньше у оставивших свой телефон в записи, чем в другой группе.

Колонка contract_period - больше клиентов, но не намного, заплативших за период 1 месяц, но отток в этой группе в несколько раз больше. В группе, заплативших за полгода, оставшихся клиентов в несколько раз больше оттока. И в группе, заплативших за 1 год почти все клиенты в числе оставшихся. 

Колонка group_visits - не любителей групповых занятий немного больше. В этой группе оттоке и оставшихся клиентов одинаковое количество. В группе любителей групповых занятий отток клиентов в несколько раз меньше остальных клиентов.

Колонка age - большая часть оставшихся клиентов, люди в возрасте в возрасте от 28 до 40 лет, здесь отток меньше всего. Отток клиентов больше всего у людей в возрасте до 28 лет.

Колонка avg_additional_charges_total - суммарная выручка от других услуг примерно в 2 раза больше у оставшися клиентов

Колонка month_to_end_contract - отток больше всего у клиентов, оформивших подписку на 1 месяц. Почти нет оттока у тех, кто купил подписку на 5, 6 и 12 месяцев.

Колонка lifetime - Отток больше всего у клиентов, посещавших клуб до 2 месяцев. А от 2 месяцев до 20 месяцев оттток значительно меньше.

Колонка Avg_class_frequency_total - средняя частота посещений в неделю за все время с начала действия абонемента примерно в 2 раза больше у оставшися клиентов.

Колонка Avg_class_frequency_current_month -  средняя частота посещений в неделю за предыдущий месяц примерно в 2 раза меньше в оттоке. 


### Матрица корреляций

In [ ]:
cm=fitness_data.corr()
cm

In [ ]:
#Построим тепловую карту по матрице корреляций
plt.figure(figsize=(35, 20))
plt.rcParams['font.size'] = '20'
sbn.heatmap(cm, annot = True, square=False)
plt.title('Тепловая карта матрицы корреляции')
plt.xticks(rotation=30)
plt.show()

Есть небольшие зависимости, меньше 0.8 между колонками partner и promo_friends, и высокая зависимость между avg_class_frequency_total и аvg_class_frequency_current_month, contract_period и month_to_end_contract, эти данные дополняют друга друга.

## Модель прогнозирования оттока клиентов

### Разбивка данных на обучающую и валидационную выборку функцией train_test_split()

In [ ]:
#Сделаем целевую переменную и матрицу признаков
X=fitness_data.drop('churn', axis=1)
y=fitness_data['churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Создание моделей логистическая регрессия и случайный лес

In [ ]:
#Логистическая регрессия
model_lr = LogisticRegression(random_state=0, solver='liblinear')
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)

In [ ]:
#Случайный лес
model_rf = RandomForestClassifier(random_state=0)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

### Оценка метрик accuracy, precision и recall для обеих моделей на валидационной выборке

In [ ]:
#Создадим функцию для расчета метрик
def print_metrics(y_test, y_pred):
    print('Метрики классификации')
    print('\tAccuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))
    print('\tPrecision: {:.2f}'.format(precision_score(y_test, y_pred)))
    print('\tRecall: {:.2f}'.format(recall_score(y_test, y_pred)))
    

In [ ]:
#Посчитаем метрики в модели логитическая регрессия
print_metrics(y_test, y_pred_lr)

In [ ]:
#Посчитаем метрики в модели случайный лес
print_metrics(y_test, y_pred_rf)

Метрики показали почти одинаковые результаты, обе модели одинаково выполняют поставленную им задачу.

## Модель прогнозирования оттока клиентов

### Стандартизация данных

In [ ]:
#Посчитаем метрики в модели логитическая регрессия
scaler = StandardScaler()
X_sc = scaler.fit_transform(X)


### Построим матрицу расстояний функцией linkage() на стандартизованной матрице признаков и нарисуем дендрограмму

In [ ]:
linked = linkage(X_sc, method = 'ward')
plt.figure(figsize=(15, 10))
dendrogram(linked, orientation='top')
plt.title('Иерархическая кластеризация клиентов')
plt.show()

На основании дендрограммы выберим 5 кластера иерархической кластеризации

### Обучение модели кластеризации на основании алгоритма K-Means 

In [ ]:
km = KMeans(n_clusters = 5, random_state=0)
labels = km.fit_predict(X_sc)

In [ ]:
#Сделаем оценку качества модели
silhouette_score(X_sc, labels) 

Качество модели не очень хорошее.

### Средние значения признаков по кластерам

In [ ]:
#Добавим колонку clusters в исходную таблицу
fitness_data['clusters']=labels

In [ ]:
#Посчитаем средние значения призанков по кластерам
data_clusters=fitness_data.groupby('clusters').mean()
data_clusters

Там где средние значения примерно одинаковые, признак не влияет на отток, где средние значения отличаются, признак влияет. 

### Распределение признаков по кластерам

In [ ]:
for i in fitness_data.columns[:-1]:
    plt.figure(figsize=(35, 15))
    plt.rcParams['font.size'] = '20'
    sbn.histplot(x=i, hue='clusters', data=fitness_data, palette="tab10")
    plt.title(f'Распределение значений признака {i} по кластерам')
    plt.ylabel(f'{i}')
    plt.show()

признак gender - заметных связей нет

признак near_location - те кто живет неблизко от клуба, кластер 2. Остальные признаки почти полностью входят в группу близко от клуба

признак partner - кластер 2 и 3 точно нет в программе партнер, остальные есть с программой партнер и без 

признак promo_friends - здесь такое же соотношение кластеров как в partner

признак phone - те кто не оставил телефон все входят в кластер 1 

признак contract_period - здесь хорошо видно, что абонементы продаются на периоды 1,6,12 месяцев. Кластер 0 состоит преимущественно из 12-месячных абонементов, и в нем нет тех у кого абонементов 1 мес и почти нет 6 месяцев

признак group_visits - в кластере 3 точно нет любителей групповых занятий. Остальные кластеры присутствуют в группе любителей групповых занятий и тех тех кто не любит

признак age - кластер 3 преимущественно состоит из тех кому 30 лет и меньше, кластер 0 кому больше 30 лет.

признак avg_additional_charges_total - здесь также кластер 3, который тратит до 250(молодежь) и кластер 0 - тратит свыше 200(старше 30).

признак month_to_end_contract - кластеры 0 и 4, те кто покупают абонемент на год, остальные меньше 2 месяцев и 2 месяца

признак lifetime - кластеры 0 и 4, те кто долго ходят в фитнесс-клуб. 3, 2 и немного 0, 1, 4, те кто ходили в клуб не больше 2 месяцев

признак avg_class_frequency_total - кластер 3, те кто ходит в среднем 1 раз, кластер 0 - те кто ходит 2-3 раза.

признак avg_class_frequency_current_month -  здесь такое же соотношение кластеров как в avg_class_frequency_total

### Доля оттока по кластерам 

In [ ]:
outflow_parts = fitness_data.query('churn==1').groupby('clusters')['churn'].count() / len(fitness_data)
outflow_parts


In [ ]:
outflow_parts=outflow_parts.append(pd.Series([0.73475]), ignore_index=True)

In [ ]:
outflow_parts

In [ ]:
outflow_parts = outflow_parts.rename(index={5:'Нет оттока'})

In [ ]:
#Нарисуем круговую диаграмму оттока по кластерам
fig = go.Figure(data=[go.Pie(labels=outflow_parts.index, values=outflow_parts.values)])
fig.update_layout(title_text='Доля оттока клиентов по кластерам')
fig.show()

Доля оттока клиентов для каждого кластера разная. Больше всего уходят клиенты из 3 кластера, 16.2% от общего числа посетителей в кластере. Самые надежные кластеры 0, 1 и 4, здесь меньше всего отток.  

## Общие выводы

Провели анализ оттока клиентов в фитнесс-клубе. Использовали для этого модели машинного обучения с учителем "Логистическая регрессия" и "Случайный лес". Обе модели одинаково показали хорошие результыты. Оценка качества модели accuracy показала даже 93%, это высокий результат. 

Так же использовали методы машинного обучения без учителя, кластеризация клиентов, при помощи иерархичсекой кластеризации и метода к-средних. Оценка качества модели показала не очень высокий результат 0,13 (максимальный результат когда показатель стремится к 1). Анализ показал, что для успешной работы фитнесс-клубе необходимо использовать следующие рекомендации:

1)привлекать клиентов предлагая партнерские программы и промо-акции
2)использовать более выгодные цены на абонементы за 6 и 12 месяцев
3)создавать приятную, притягивающую атмосферу, чтобы клиенты ходили чаще 2-3 раза в неделю
4)больше уделять внимание людям возрастом 28 лет старше, это потенциально надежные клиенты.